In [ ]:
from imutils import paths
import cv2
import os
from tqdm import tqdm_notebook
import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

In [ ]:
imagePaths = list(paths.list_images("/content/Covid-19-20240229T060836Z-001.zip, /content/Non Covid-19-20240229T060836Z-001.zip"))
data = []
labels = []

In [ ]:
img_height=192
img_width = 192
num_classes = 2

In [ ]:
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the image, swap color channels, and resize it to be a fixed
	# 192x192 pixels while ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (img_height, img_width))

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)


In [ ]:

import numpy as np
data = np.array(data) / 255.0
labels = np.array(labels)


In [ ]:
# perform one-hot encoding on the labels
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical

# Define file paths
file_paths = ["/content/Covid-19-20240229T060836Z-001.zip", "/content/Non Covid-19-20240229T060836Z-001.zip"]

# Extract labels from file paths
labels = [os.path.basename(file_path).split()[0] for file_path in file_paths]

# Perform one-hot encoding on the labels
lb = LabelBinarizer()
labels_encoded = lb.fit_transform(labels)
labels_one_hot = to_categorical(labels_encoded)

print("Original labels:", labels)
print("Encoded labels:", labels_encoded)
print("One-hot encoded labels:", labels_one_hot)


Original labels: ['Covid-19-20240229T060836Z-001.zip', 'Non']
Encoded labels: [[0]
 [1]]
One-hot encoded labels: [[1. 0.]
 [0. 1.]]


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'data' is your input data and 'labels' are your corresponding labels
# Ensure that 'data' and 'labels' have the same length or shape

# Example: Creating dummy data
data = np.random.rand(100, 32, 32, 3)  # Assuming input data shape is (100, 32, 32, 3)
labels = np.random.randint(0, 2, 100)  # Assuming binary labels for 100 samples

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels,
                                                    test_size=0.20,
                                                    stratify=labels,
                                                    random_state=40)

# Displaying shapes of the resulting datasets
print("Number of training examples:", X_train.shape[0])
print("Number of test examples:", X_test.shape[0])
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)



Number of training examples: 80
Number of test examples: 20
X_train shape: (80, 32, 32, 3)
y_train shape: (80,)
X_test shape: (20, 32, 32, 3)
y_test shape: (20,)


In [ ]:
from keras import applications

base_model = applications.resnet50.ResNet50(
                            weights= 'imagenet',
                            include_top=False,
                            input_shape= (img_height,img_width,3))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
from keras.models import Sequential,Model,load_model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation= 'relu')(x)

model = Model(inputs= base_model.input, outputs= predictions)

In [ ]:
from keras.optimizers import SGD, Adam
#sgd = SGD(lr=0.0001, momentum=0.9, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from skimage.transform import resize
from sklearn.model_selection import train_test_split

# Assuming you have loaded your data into X_train and y_train
# Assuming X_train has shape (num_samples, 32, 32, 3)
# Assuming y_train has shape (num_samples, num_classes)

# Resize the input data to match the expected input shape of the model (192, 192, 3)
X_train_resized = np.array([resize(image, (192, 192, 3)) for image in X_train])

# Define your CNN model
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(192, 192, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # Assuming num_classes is the number of output classes
    ])
    return model

# Create your model
model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Split the data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_resized, y_train, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train_split, y_train_split, epochs=100, batch_size=10, validation_data=(X_val_split, y_val_split))

# Evaluate the model
loss, accuracy = model.evaluate(X_val_split, y_val_split)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


Epoch 1/100
7/7 [==============================] - 4s 151ms/step - loss: 1.5146 - accuracy: 0.4844 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 2/100
7/7 [==============================] - 0s 18ms/step - loss: 0.6778 - accuracy: 0.6094 - val_loss: 0.7055 - val_accuracy: 0.5000
Epoch 3/100
7/7 [==============================] - 0s 20ms/step - loss: 0.6595 - accuracy: 0.6250 - val_loss: 0.7382 - val_accuracy: 0.5000
Epoch 4/100
7/7 [==============================] - 0s 18ms/step - loss: 0.5702 - accuracy: 0.6719 - val_loss: 0.7679 - val_accuracy: 0.5000
Epoch 5/100
7/7 [==============================] - 0s 20ms/step - loss: 0.4653 - accuracy: 0.7969 - val_loss: 0.7291 - val_accuracy: 0.5000
Epoch 6/100
7/7 [==============================] - 0s 20ms/step - loss: 0.5278 - accuracy: 0.7656 - val_loss: 0.7309 - val_accuracy: 0.5000
Epoch 7/100
7/7 [==============================] - 0s 18ms/step - loss: 0.3659 - accuracy: 0.9688 - val_loss: 0.7549 - val_accuracy: 0.6250
Epoch 8/100
7/7 [==

In [ ]:
# Assuming you have already imported necessary libraries and defined your model

# Resize the test images to match the expected input shape of the model
X_test_resized = np.array([resize(image, (192, 192, 3)) for image in X_test])

# Normalize the pixel values of the resized images
X_test_resized = X_test_resized.astype('float32') / 255.0

# Evaluate the model using the preprocessed test data
preds = model.evaluate(X_test_resized, y_test)

# Print the evaluation results
print("Loss =", preds[0])
print("Test Accuracy =", preds[1])

1/1 [==============================] - 0s 397ms/step - loss: 0.6771 - accuracy: 0.6000
Loss = 0.6770865321159363
Test Accuracy = 0.6000000238418579


In [ ]:
from sklearn.metrics import classification_report

# Resize the test images to match the expected input shape of the model
X_test_resized = np.array([resize(image, (192, 192, 3)) for image in X_test])

# Normalize the pixel values of the resized images
X_test_resized = X_test_resized.astype('float32') / 255.0

# Make predictions on the preprocessed test data
predIdxs = model.predict(X_test_resized, batch_size=10)

# Convert the predictions to class labels
predIdxs = np.argmax(predIdxs, axis=1)

# Generate a classification report directly using y_test and predIdxs
print(classification_report(y_test, predIdxs, target_names=lb.classes_))


2/2 [==============================] - 0s 6ms/step
                                   precision    recall  f1-score   support

Covid-19-20240229T060836Z-001.zip       0.00      0.00      0.00         8
                              Non       0.60      1.00      0.75        12

                         accuracy                           0.60        20
                        macro avg       0.30      0.50      0.37        20
                     weighted avg       0.36      0.60      0.45        20



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 190, 190, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 95, 95, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 93, 93, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 46, 46, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 44, 44, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 123904)            0